# 学認連携の設定を行う -- 直接学認フェデレーションを利用する

---

学認と連携する設定を行います。IdP-proxyを利用せずにCoursewareHubを直接SPとして学認に登録します。

## 概要

このNotebookで設定するCoursewareHubと学認フェデレーションの構成要素を以下に示します。

![構成](images/cw-321-01.png)

このNotebookでは IdP-proxyを利用せずに直接CoursewareHubをSPとして学認に登録する手順について記しています。

各コンテナの役割や処理の詳細については https://github.com/NII-cloud-operation/CoursewareHub-LC_jupyterhub-deploy を参照してください。

### 前提条件

このNotebookは構築済みのCoursewareHubに対して学認連携による認証を追加する設定を行います。そのためCoursewareHubが構築済みであることを前提としています。

### おもな設定手順

このNotebookの主な設定手順を以下に示します。

1. mAPグループの作成
1. CoursewareHubに対して学認連携に関する設定変更を行う
1. 学認にSP設置の申請を行う
1. 学認mAPとの連携


このNotebookでの設定手順は学認との連携を設定するためCoursewareHubの設定だけではなく学認のシステムでの設定手順も含まれています。上に示した手順のうち　1　と 4 は学認のクラウドゲートウェイでの設定となります。また手順 3 は学認申請システムでの設定となります。手順2がCoursewareHubに対する設定となっています。

学認に関する設定手順ではウェブインターフェースによる設定だけでなく、メールなどによる学認側への依頼が必要となることがあります。

### UnitGroup名

学認連携を設定する対象となるUnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

### チェック

指定されたUnitGroup名が前提条件を満たしていることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

構築対象となるVCノードにアクセスできることを確認します。

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

UnitGroup名に対応する group_varsファイルが存在していることを確認します。

In [ ]:
!test -f group_vars/{ugroup_name}

CoursewareHubのサービスが実行されていることを確認します。

In [ ]:
cmd = "docker stack services --format '{{.Replicas}}' coursewarehub | grep '1/1' | wc -l"
!ansible {target_hub} -m shell -a "{{% raw %}} [ \$({cmd}) -eq 4 ] {{% endraw %}}"

## mAPグループの作成

[学認クラウドゲートウェイサービス](https://cg.gakunin.jp/)またはSP検証環境(テストフェデレーションの場合)にアクセスしてグループの作成などを行います。手順の詳細については「[GakuNin mAPマニュアル--グループ管理者](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=20873726)」を参照してください。

テストフェデレーションでクラウドゲートウェイの機能（SP検証環境）を利用するには事前に利用申請が必要となります。学認mAPの「[問い合わせ](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=8716731)」などに記されている窓口を通してSP検証環境を利用するための依頼を行ってください。テストフェデレーションSP検証環境のアドレスは利用申請後にメールなどで通知されます。

### グループの作成

学認クラウドゲートウェイのウェブサイトから「グループの作成」を実行してください。

![初期画面](images/cw-211-01.png)

> キャプチャー画面はテストフェデレーションのものです。

グループの作成画面に表示された項目を入力してください。

![グループの作成](images/cw-211-02.png)


後ほどCoursewareHubに対して学認連携設定を行う際に、ここで「グループID」として指定した値が必要となります。

### メンバーの招待

必要に応じて「メンバー招待」からCoursewareHubの利用者に対してグループへの招待を行ってください。

![メンバー招待](images/cw-211-07.png)

## CoursewareHubの設定

CoursewareHubに対して学認連携設定を行います。

### パラメータを指定する

CoursewareHubと連携する学認フェデレーションに関するパラメータを設定します。

#### フェデレーション

参加するフェデレーションがテストフェデレーションであるか否かを次のセルで指定してください。

In [ ]:
# (例)
# enable_test_federation = False   # 運用フェデレーション
# enable_test_federation = True    # テストフェデレーション

enable_test_federation = 

#### DSサーバ

DSのサーバ名を指定します。

指定しない場合は運用フェデレーションのDSサーバ名がデフォルト値として用いられます。テストフェデレーションに参加している場合は次のセルのコメントを外してDSサーバ名を指定してください。

In [ ]:
# gakunin_ds_hostname = 'test-ds.gakunin.nii.ac.jp'    # テストフェデレーション

#### クラウドゲートウェイ
クラウドゲートウェイのサーバ名を指定します。

指定しない場合は運用フェデレーションのサーバ名がデフォルト値として用いられます。テストフェデレーションに参加している場合は次のセルのコメントを外してクラウドゲートウェイのサーバ名を指定してください。

In [ ]:
# cg_fqdn = 'sptest.cg.gakunin.jp'    # テストフェデレーション

#### メタデータ検証用証明書

メタデータを検証するための証明書を指定します。

証明書を配布しているURLを次のセルで指定してください。

In [ ]:
# (例)
# metadata_signer_url = 'https://metadata.gakunin.nii.ac.jp/gakunin-signer-2017.cer'          # 運用フェデレーション
# metadata_signer_url = 'https://metadata.gakunin.nii.ac.jp/gakunin-test-signer-2020.cer'  # テストフェデレーション

metadata_signer_url = 

####  mAPのグループを指定する

CoursewareHubの利用を許可対象となる利用者を指定します。

指定する値は「[2. mAPグループの作成](#mAPグループの作成)」で作成した学認mAPのグループIDとなります。グループIDの指定はリストの形式で行うので複数のグループIDを指定することができます。

In [ ]:
# (例)
# cg_groups = [
#    'group1-id',
#    'group2-id',
#]

cg_groups = [
    
]

#### パラメータの保存

ここまで指定したパラメータを ansible の変数として `group_vars`ファイルに保存します。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)

cg_group_prefix = (
    "https://cg.gakunin.jp/gr/"
    if not enable_test_federation
    else "https://sptest.cg.gakunin.jp/gr/"
)

for key in ["auth_fqdn"]:
    if key in gvars:
        del(gvars[key])

gvars.update({
    'enable_federation': True,
    'enable_test_federation': enable_test_federation,
    'metadata_signer_url': metadata_signer_url,
    'cg_groups': [
        f'{cg_group_prefix}{x}' if not x.startswith('https://') else x
        for x in cg_groups
    ],
})

jupyterhub_param_names = ['gakunin_ds_hostname', 'cg_fqdn']
for name in jupyterhub_param_names:
    if name in vars():
        gvars[name] = vars()[name]
    elif 'name' in gvars:
        del(gvars['name'])

with gvars_path.open(mode='w') as f:
    yaml.safe_dump(gvars, stream=f)
    
!cat group_vars/{ugroup_name}

### 設定ファイルの配置、更新

学認連携のために必要となる設定ファイルの配置や更新を行います。

ここで配置、更新するファイルを以下の表に示します。

|パス|用途|
|:---|:---|
|/etc/jupyterhub/simplesamlphp/cert/gakunin-signer.cer|メタデータ検証用証明書|
|/etc/jupyterhub/hub-const.php|auth-proxyコンテナの設定ファイル(更新)|
|/opt/coursewarehub/docker-compose.yml|docker-composeの設定ファイル(更新)|


まず実際に設定を変更する前にドライラン（チェックモード）でansibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/setup-gakunin-sp.yml || true

実際に設定変更を行います。

In [ ]:
!ansible-playbook -l {target_hub} playbooks/setup-gakunin-sp.yml

配置したファイルを確認します。

In [ ]:
!ansible {target_hub} -a 'tree -F {{{{jupyterhub_cfg_dir}}}}'

### コンテナの更新

コンテナを起動し直して変更した設定の反映を行います。

起動中のコンテナを一度停止します。

In [ ]:
!ansible {target_hub} -a 'docker stack rm coursewarehub'

コンテナを起動し直します。

In [ ]:
!ansible {target_hub} -a 'chdir={{{{compose_dir}}}} \
    docker stack deploy -c docker-compose.yml coursewarehub'

全てのコンテナが起動するまで待ち合わせを行います。


In [ ]:
import time
import sys


cmd = "docker stack services --format '{{.Replicas}}' coursewarehub | grep '1/1' | wc -l"
for retry in range(18):
    time.sleep(10)
    try:
        !ansible {target_hub} -m shell -a "{{% raw %}} [ \$({cmd}) -eq 4 ] {{% endraw %}}"
        break
    except:
        print('retry', file=sys.stderr)
else:
    !ansible {target_hub} -a 'docker stack services coursewarehub'
    raise RuntimeError("起動処理が完了しませんでした。")

コンテナの起動状態を確認します。

In [ ]:
!ansible {target_hub} -a 'docker stack services coursewarehub'

## 学認にSP設置の申請を行う

申請を行う前に学認（GakuNin）の「[参加情報](https://www.gakunin.jp/join)」にてフェデレーション参加の流れを確認してください。参加するフェデレーションに従い「[テストフェデレーション参加手続き](https://www.gakunin.jp/join/test)」または「[運用フェデレーション参加手続き](https://www.gakunin.jp/join/production)」にある「学認申請システム」から「新規SP申請」を行います。

「新規SP申請」を選択すると以下のような画面が表示されます。
> キャプチャー画面はテストフェデレーションのものです。

![新規SP申請](images/cw-531-01.png)

CoursewareHubでは[SimpleSAMLphp](https://simplesamlphp.org/)を利用しています。そのため学認申請システムのテンプレートではなく、構築したCoursewareHub のメタデータを「テンプレート外メタデータ」からアップロードして申請してください。

![メタデータアップロード](images/cw-531-02.png)

CoursewareHubからメタデータをダウンロードします。次のセルを実行することで表示されるリンク先にアクセスするとメタデータがダウンロードされます。

In [ ]:
print(f'https://{gvars["master_fqdn"]}/simplesaml/module.php/saml/sp/metadata.php/default-sp')

ダウンロードしたメタデータを学認申請システムにアップロードすると「SPメタデータ情報」の入力欄のうちメタデータに対応するものが設定されます。他の欄を入力した後に申請してください。ただし CoursewareHubでは `mail` 属性を利用するので「受信する属性情報」に `mail` を**必須** な項目として追加してください。

## 学認mAPとの連携

CoursewareHubでは利用者をグループ管理するために[学認mAP](https://meatwiki.nii.ac.jp/confluence/display/gakuninmappublic/Home)を利用します。ここでは CoursewareHub を学認mAPと連携する手順について記します。

### 申請

mAPとの連携を行うためには、構築したCoursewareHubをSPコネクタとしてmAPに登録します。SPコネクタの登録を行うには事前にSP管理者権限を学認側のシステムで付与してもらう必要があります。学認mAPの「[問い合わせ](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=8716731)」などに記されている窓口を通して「SPコネクタの作成」を行うため設定を依頼してください。

### メタデータの登録（テストフェデレーションの場合）

テストフェデレーションの場合、SP検証環境(mAP)のメタデータファイルをCoursewareHubに配置します。運用フェデレーションの場合はこの手順は不要です。

SP検証環境利用申請を行うと、学認クラウドゲートウェイサービスサポートからSP検証環境のメタデータが送られてきます。送られてきたメタデータを CoursewareHub に登録します。メタデータファイルをこのNotebook環境に配置して、そのパスを次のセルで指定してください。

In [ ]:
# (例)
# cgidp_metadata = 'sptestcgidp-metadata-20200918.xml'

cgidp_metadata = 

指定したパラメータをgroup varsファイルに保存します。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)
    
if gvars["enable_test_federation"]:
    gvars.update({"cgidp_metadata": str(Path(cgidp_metadata).resolve())})
    with gvars_path.open(mode='w') as f:
        yaml.safe_dump(gvars, stream=f)

!cat group_vars/{ugroup_name}

SP検証環境のメタデータをCoursewareHubの環境に配置します。ここで配置するファイルを以下の表に示します。

|パス|用途|
|:---|:---|
|/etc/jupyterhub/simplesamlphp/metadata/xml/cgidp-metadata.xml|SP検証環境のメタデータ|


まず実際に設定を変更する前にドライラン（チェックモード）でansibleを実行します。

In [ ]:
!ansible-playbook -l {target_hub} -CDv playbooks/setup-gakunin-sp.yml || true

実際に設定変更を行います。

In [ ]:
!ansible-playbook -l {target_hub} playbooks/setup-gakunin-sp.yml

配置したファイルを確認します。

In [ ]:
!ansible {target_hub} -a 'tree -F {{{{jupyterhub_cfg_dir}}}}'

### SPコネクタの作成

[学認クラウドゲートウェイサービス](https://cg.gakunin.jp/)または(テストフェデレーションの場合)SP検証環境にアクセスしてSPコネクタの作成を行います。手順の詳細については学認クラウドゲートウェイサービス利用マニュアルの「[SPコネクタを作成する](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=20873902)」の章を参照してください。

SP管理者が学認クラウドゲートウェイのウェブサイトにアクセスすると以下のような画面が表示されます。

> キャプチャー画面はテストフェデレーションのものです。

![クラウドゲートウェイ画面](images/cw-531-03.png)

「SPコネクタの作成」を選択すると入力画面が表示されるので、必要な項目を入力してください。

![新規SPコネクタ入力画面](images/cw-531-04.png)

詳細設定の「Q1 作成するSPコネクタは、検索対象にしますか？」との項目で「検索対象としない」を選択した場合は、グループ管理者がSPコネクタを見つけることが出来なくなります。グループとの接続を行うためにはSPコネクタ側からグループへ接続申請をする必要があります。

### グループをSPコネクタに接続する

「[2. mAPグループの作成](#mAPグループの作成)」で作成したグループをサービス(SPコネクタ)に紐づけ、グループに属するユーザがCoursewareHubサービス(SP)を利用できるようにします。

> ここでは SPコネクタの管理者とグループの管理者が同一であることを想定しています。

グループとサービス（SPコネクタ）の紐づけは、グループ側からサービスを指定する方法とサービス側（SPコネクタ）からグループを指定して接続する方法があります。ここではグループ管理者がSPコネクタ（サービス）へと接続申請を行う手順を示します。設定方法の詳細については学認クラウドゲートウェイマニュアルの「[サービスを利用する](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=20873789)」を参照してください。また、逆の関連付けとなるSPコネクタからグループ管理者へ接続申請を行う手順については「[SPコネクタにグループを接続する](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=20873907)」を参照してください。

まず、グループの管理者メニューから「利用Webサービス」を選択します。

![グループ画面](images/cw-211-03.png)

「利用するWebサービスを追加」ボタンを押すと、サービスの一覧が表示されるので「3.3 SPコネクタの作成」で登録したサービスを選択してください。

> SPコネクタとして登録する際に、詳細設定の「Q1 作成するSPコネクタは、検索対象にしますか？」の値を「検索対象としない」と設定した場合はWebサービスの一覧には表示されません。

![サービス一覧画面](images/cw-211-04.png)

CoursewareHubのサービスを選択して申請を行うと承認待ちの状態となります。

![承認待ち画面](images/cw-211-05.png)

SPコネクタの設定画面から承認を行うと申請状態が「承認」となります。

![グループ画面](images/cw-211-06.png)

## CoursewareHubに学認IdPのアカウントでログインする

学認IdPのアカウントでCoursewareHubにログインできることを確認します。

次のセルを実行すると、CoursewareHubのアドレスを表示します。

In [ ]:
print(f'https://{gvars["master_fqdn"]}')

ログイン画面に表示されている「学認フェデレーションへ」のリンクから学認IdPを利用したログインができます。

![ログイン画面](images/cw-221-02.png)

「学認フェデレーションへ」のリンクから進むと以下のようなIdPの選択画面が表示されます。

> キャプチャー画面はテストフェデレーションのものです。

![IdP選択画面](images/cw-221-03.png)

IdPを選択しログインを行ってください。利用を許可したmAPグループに所属しているユーザであればsingle-userサーバが実行されてJupyter環境を利用することができます。